In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import math

In [2]:
# Base Equations
def delimiter(val):
    if (val > 100):
        return 100
    elif (val < 0):
        return 0
    else:
        return val

# y = x^2 + Ax + B
def y_poly2(p, a, b, c):
    return a*(p**2) + (b * p) + c

# y = x^4 + Ax^3 + Bx^2 + Cx + D
def y_poly4(p, a, b, c, d, e):
    return a*(p**4) + b*(p**3) + c*(p**2) + (d * p) + e

# y = Ax^B
def y_exp(p, a, b):
    return a*(p**b)

# y = Ax + B
def y_poly(p, a, b):
    return (a * p) + b

In [3]:
# EQS
# Temperature and Relative Humidity EQS
def calculate_wet_bulb_temp(temp, rh):
    return temp * math.atan(1.51977e-01*(rh + 8.313659)**(0.5)) + math.atan(temp + rh) - math.atan(rh - 1.676331) + 3.91838e-03*((rh)**(1.5))*math.atan(2.3101e-02*rh) - 4.686035

def get_temp_eqs(temp, rh):
    tw = calculate_wet_bulb_temp(temp, rh)
    return delimiter(y_poly4(tw,-1.23709262e-03, 6.47697081e-02, -1.20204332e-01, -2.73184189e+01, 2.82805840e+02))

# CO EQS
def calculate_poor_co_threshold(t):
    return y_exp(t, 6.61549908e+04, -2.13976492e+00)
    
def get_co_eqs(co, t):
    co_thres = calculate_poor_co_threshold(t)
    return delimiter((75 * co)/co_thres)

# CO2 EQS
def get_co2_eqs(co2):
    return delimiter(y_poly2(co2, -5.05243336e-06, 4.33185708e-02, 9.62634873e+00))

# UV EQS
def get_uv_eqs(uv):
    return uv * 10

# NO2 EQS
def get_no2_eqs(no2):
    return delimiter(y_poly2(no2, -9.89795919e+01, 5.09897959e+02, 0))

# O3 EQS
def calculate_poor_o3_threshold(t):
    return y_poly(t, -1.28571429e-04, 1.32714286e-01)
    
def get_o3_eqs(o3, t):
    o3_thres = calculate_poor_o3_threshold(t)
    return delimiter((75 * o3)/o3_thres)

# PM EQS
def calculate_pm_scaled(pm, t):
    return pm * (1440/t)

def get_pm25_eqs(pm, t):
    pm_scaled = calculate_pm_scaled(pm, t)
    return delimiter(y_poly2(pm_scaled, -7.21522673e-04, 4.74950569e-01, 2.19025801e+01))

def get_pm10_eqs(pm, t):
    pm_scaled = calculate_pm_scaled(pm, t)
    return delimiter(y_poly2(pm_scaled, -4.69422877e-05, 1.69665921e-01, 1.54747566e+01))

# TVOC EQS
def get_tvoc_eqs(tvoc):
    return delimiter(y_poly2(tvoc, -21.32720492, 96.78504677, 1.48876859))

# Noise EQS (db) developed but not used
def get_noise_eqs(noise):
    return delimiter(y_poly(noise, 0.875, -2.5))

# Ethanol EQS
def get_eth_eqs(eth):
    return delimiter(y_poly(eth, 0.1, 0))

# Average EQS
def get_avg_eqs(t, temp, rh, co, co2, uv, no2, o3, pm25, pm10, tvoc, eth):
    temp_eqs = get_temp_eqs(temp, rh)
    co_eqs = get_co_eqs(co, t)
    co2_eqs = get_co2_eqs(co2)
    uv_eqs = get_uv_eqs(uv)
    no2_eqs = get_no2_eqs(no2)
    o3_eqs = get_o3_eqs(o3, t)
    pm25_eqs = get_pm25_eqs(pm25, t)
    pm10_eqs = get_pm10_eqs(pm10, t)
    tvoc_eqs = get_tvoc_eqs(tvoc)
    eth_eqs = get_eth_eqs(eth)
    
    return (1/10)* (temp_eqs + co_eqs + co2_eqs + uv_eqs + no2_eqs + o3_eqs + pm25_eqs + pm10_eqs + tvoc_eqs + eth_eqs)

In [5]:
# get_avg_eqs vars
# t, temp, rh, co, co2, uv, no2, o3, pm25, pm10, tvoc, eth

# Parking Lot
pl_eqs = get_avg_eqs(130, 10.65, 66, 10.96, 854, 2, 31.49, 6.21, 6.78, 7.78, 19.59, 192.24)
print("Parking Lot EQS:", pl_eqs)

# SU Atrium
sa_eqs = get_avg_eqs(190, 20.05, 45, 10.88, 934, 2, 33.88, 6.20, 4.5, 4.7, 23.40, 188.34)
print("SU Atrium:", sa_eqs)

# Dormitory
dm_eqs = get_avg_eqs(577, 24, 56, 3.55, 2068, 1.8, 43.09, 5.23, 0.1, 0.1, 103, 180.11)
print("Dormitory:", dm_eqs)

# Living Room
lr_eqs = get_avg_eqs(1127, 15, 66, 8.17, 824, 1.94, 22.68, 5.35, 6.6, 7.5, 1193, 190.17)
print("Living Room:", dm_eqs)

# Rocky Terrace
rt_eqs = get_avg_eqs(1736, 14.68, 65, 8.57, 798, 1.94, 21.55, 5.78, 8.06, 9.30, 1051, 190.31)
print("Rocky Terrace Room:", rt_eqs)

# Engineering Basement
eb_eqs = get_avg_eqs(7225, 25.78, 34, 10.71, 619, 2.02, 18.69, 6, 2.34, 3.91, 245, 188.93)
print("Engineering Basement:", eb_eqs)

# March 2nd
mar_eqs = get_avg_eqs(73, 21.59, 34, 10.62, 3585, 2.27, 24.41, 5.99, 0.2, 0.32, 10.19, 189.81)
print("March 2nd:", mar_eqs)


Parking Lot EQS: 46.540980149084646
SU Atrium: 35.70278741508944
Dormitory: 35.12182802872686
Living Room: 35.12182802872686
Rocky Terrace Room: 26.005332451791784
Engineering Basement: 21.13285272858818
March 2nd: 39.91546092823609
